In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
from gensim.models.doc2vec import Doc2Vec, LabeledSentence 
import numpy as np
import copy
%matplotlib inline

In [2]:
def load_data():
    train_ori = pd.read_table("round1_ijcai_18_train_20180301.txt",delim_whitespace=True)
    test_ori = pd.read_table('round1_ijcai_18_test_a_20180301.txt',delim_whitespace = True)
    train_ori = train_ori.drop_duplicates()
    train_ori = train_ori.reset_index(drop = True)
    test_ori = test_ori.drop_duplicates()
    test_ori = test_ori.reset_index(drop = True)
    return train_ori, test_ori

In [3]:
train_ori, test_ori = load_data()
train_1 = train_ori.copy(deep = True)
test_1 = test_ori.copy(deep = True)

In [4]:
test_ori.shape

(18371, 26)

In [5]:
train_ori.shape

(478111, 27)

In [6]:
train_y = train_ori["is_trade"]
train_ori = train_ori.drop("is_trade",axis = 1)

In [7]:
train_ori.shape

(478111, 26)

In [8]:
data = pd.concat([train_ori,test_ori])
data = data.reset_index(drop = True)
data.shape

(496482, 26)

In [9]:
data.isnull().sum()

instance_id                  0
item_id                      0
item_category_list           0
item_property_list           0
item_brand_id                0
item_city_id                 0
item_price_level             0
item_sales_level             0
item_collected_level         0
item_pv_level                0
user_id                      0
user_gender_id               0
user_age_level               0
user_occupation_id           0
user_star_level              0
context_id                   0
context_timestamp            0
context_page_id              0
predict_category_property    0
shop_id                      0
shop_review_num_level        0
shop_review_positive_rate    0
shop_star_level              0
shop_score_service           0
shop_score_delivery          0
shop_score_description       0
dtype: int64

# 处理item_category_list这个feature

In [10]:
data

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,context_timestamp,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1537236544,4006,5799347067982556520:-1;509660095530134768:-1;5...,6765930309048922341,4,1.000000,5002,1.000000,1.000000,1.000000
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1537243232,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.000000,5002,1.000000,1.000000,1.000000
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1537211052,4001,5799347067982556520:5131280576272319091;725801...,6765930309048922341,4,1.000000,5002,1.000000,1.000000,1.000000
3,937088850059189027,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1537222670,4016,509660095530134768:-1;5799347067982556520:-1;7...,6765930309048922341,4,1.000000,5002,1.000000,1.000000,1.000000
4,7975697065017708072,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1537271320,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.000000,5002,1.000000,1.000000,1.000000
5,7764762765372067286,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1537282855,4003,5799347067982556520:9172976955054793469;871073...,6765930309048922341,4,1.000000,5002,1.000000,1.000000,1.000000
6,6956333474094867789,285660928590172217,7908382889764677758;8277336076276184272,2072967855524022579;5131280576272319091;263639...,9057103201734987852,548352491538518780,8,9,8,13,...,1537280317,4001,2011981573061447208:7199361004668592209;827733...,4885989684392199728,15,0.985427,5012,0.974878,0.976863,0.969278
7,8387099821892927911,285660928590172217,7908382889764677758;8277336076276184272,2072967855524022579;5131280576272319091;263639...,9057103201734987852,548352491538518780,8,9,8,13,...,1537261120,4001,2011981573061447208:7199361004668592209;827733...,4885989684392199728,15,0.985427,5012,0.974878,0.976863,0.969278
8,4021878205550012615,5202355029344881809,7908382889764677758;5755694407684602296,2072967855524022579;5131280576272319091;263639...,5520678735822176314,548352491538518780,8,9,10,16,...,1537208871,4001,5755694407684602296:-1;5799347067982556520:-1;...,4885989684392199728,15,0.985427,5012,0.974878,0.976863,0.969278
9,6499571365974135517,285660928590172217,7908382889764677758;8277336076276184272,2072967855524022579;5131280576272319091;263639...,9057103201734987852,548352491538518780,8,9,8,13,...,1537285390,4003,8277336076276184272:820214312075361939;5096600...,4885989684392199728,15,0.985427,5012,0.974878,0.976863,0.969278


In [11]:
cate_list = list(data['item_category_list'])

In [12]:
#分号前的category1
cate_1_list = []
for i in range(len(cate_list)):
    cate_1_list.append(cate_list[i].split(";")[0])

In [13]:
set(cate_1_list)#发现所有样本都一样 因此去掉该子字段的信息

{'7908382889764677758'}

In [14]:
#分号后的category2
cate_2_list = []
for i in range(len(cate_list)):
    cate_2_list.append(cate_list[i].split(";")[1])

In [15]:
set(cate_2_list)#有区别 因此用该子字段的信息代替原来的feature

{'1968056100269760729',
 '2011981573061447208',
 '22731265849056483',
 '2436715285093487584',
 '2642175453151805566',
 '3203673979138763595',
 '4879721024980945592',
 '509660095530134768',
 '5755694407684602296',
 '5799347067982556520',
 '7258015885215914736',
 '8277336076276184272',
 '8710739180200009128'}

In [16]:
data["item_category_list"] = cate_2_list

In [17]:
data.head(3)

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,context_timestamp,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description
0,108641074714126964,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1537236544,4006,5799347067982556520:-1;509660095530134768:-1;5...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0
1,5754713551599725161,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1537243232,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0
2,842679481291040981,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1537211052,4001,5799347067982556520:5131280576272319091;725801...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0


In [18]:
data.shape

(496482, 26)

# 处理item_property_list这个feature

In [19]:
property_list = list(data['item_property_list'])

In [20]:
#分号分割后的所有属性放到一起
property_list_all = []
for i in range(len(property_list)):
    for j in range(len(property_list[i].split(";"))):
        property_list_all.append(property_list[i].split(";")[j])
#看看有多少种不同的属性
property_distinct = list(set(property_list_all))
print(len(property_distinct))

In [21]:
#将原始的数据转换成doc2vec可用的lists of list的格式
property_list_pre = []
for i in range(len(property_list)):
    temp = []
    temp.append(property_list[i])
    property_list_pre.append(temp)
pre_process = [[word for word in document[0].split(';')] for document in property_list_pre]

In [22]:
#看看拆分结果
print(property_list_pre[:2])
print("-"*50)
print(pre_process[:2])

[['2072967855524022579;5131280576272319091;2636395404473730413;8070395809181082496;6434796455031995313;7839592306500064003;3657871859501171040;1782439090818545916;6936314038707413523;5117290135792467652;720840888466250585;8838717711606756251;9172976955054793469;6296485275518770064;8154150695030794787;8199832121759092112;773321667872991333;7270669313837600482;3408398779125901630;4359277201620147874;448184584262668803;9029095803741432716'], ['2072967855524022579;5131280576272319091;2636395404473730413;8070395809181082496;6434796455031995313;7839592306500064003;3657871859501171040;1782439090818545916;6936314038707413523;5117290135792467652;720840888466250585;8838717711606756251;9172976955054793469;6296485275518770064;8154150695030794787;8199832121759092112;773321667872991333;7270669313837600482;3408398779125901630;4359277201620147874;448184584262668803;9029095803741432716']]
--------------------------------------------------
[['2072967855524022579', '5131280576272319091', '263639540447373

In [23]:
def doc2vec(doc, size, window = 1, min_count = 1):
    """
    doc:lists of list格式
    size:目标vector大小 
    """
    TaggededDocument = gensim.models.doc2vec.TaggedDocument  
    labeled_doc = []  
    for i, text in enumerate(doc):  
        document = TaggededDocument(text, tags=[i])  
        labeled_doc.append(document)
    model_dm = Doc2Vec(labeled_doc, min_count = min_count, window = window, size = size, 
                       negative=5, workers=4)
    array_property = model_dm.infer_vector(pre_process[0])
    for i in doc[1:]:
        temp = model_dm.infer_vector(i)
        array_property = np.vstack((array_property,temp))
    dataframe_property = pd.DataFrame(array_property)
    return dataframe_property

In [24]:
dataframe_property = doc2vec(pre_process, 10)
dataframe_property.head(3)

,0,1,2,3,4,5,6,7,8,9
0,0.037733,0.052614,0.253060,-0.023049,0.001877,-0.199947,-0.046342,0.088241,0.014969,-0.085267
1,-0.108501,-0.040940,0.412249,-0.241337,0.067854,-0.016272,0.050968,0.010247,0.236364,-0.291694
2,-0.004048,0.255547,0.259327,0.166099,-0.215297,0.193300,-0.139771,-0.211507,-0.057105,-0.248570


In [26]:
dataframe_property.shape

(496482, 10)

In [27]:
data.shape

(496482, 26)

In [25]:
dataframe_property.columns = ["prop1","prop2","prop3","prop4","prop5",
                             "prop6","prop7","prop8","prop9","prop10",]

In [28]:
data_1 = pd.concat([data, dataframe_property],axis=1)

In [29]:
data_1 = data_1.drop('item_property_list',axis = 1)
data_1.head(3)

,instance_id,item_id,item_category_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,...,prop1,prop2,prop3,prop4,prop5,prop6,prop7,prop8,prop9,prop10
0,108641074714126964,3412720377098676069,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,4505772604969228686,...,0.037733,0.052614,0.253060,-0.023049,0.001877,-0.199947,-0.046342,0.088241,0.014969,-0.085267
1,5754713551599725161,3412720377098676069,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,2692638157208937547,...,-0.108501,-0.040940,0.412249,-0.241337,0.067854,-0.016272,0.050968,0.010247,0.236364,-0.291694
2,842679481291040981,3412720377098676069,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,5247924392014515924,...,-0.004048,0.255547,0.259327,0.166099,-0.215297,0.193300,-0.139771,-0.211507,-0.057105,-0.248570


In [30]:
data_1.shape

(496482, 35)

# 处理predict_category_property这个feature

In [31]:
pred_cate_prop_list = list(data_1['predict_category_property'])

In [32]:
pred_cate_prop_list[:3]

['5799347067982556520:-1;509660095530134768:-1;5755694407684602296:-1;8277336076276184272:9148482949976129397;7908382889764677758:-1',
 '5799347067982556520:9172976955054793469;7908382889764677758:1787573075717641245,9172976955054793469,5195139481388729954,4621934203383159480,9148482949976129397',
 '5799347067982556520:5131280576272319091;7258015885215914736:5131280576272319091;7908382889764677758:5131280576272319091']

In [33]:
#用；分解每个cate：property
pcp_list = []
for i in pred_cate_prop_list:
    pcp_list.append(i.split(';'))
#再用：将cate，property分开
pcp_list_2 = []
for i in pcp_list:
    tmp = []
    for k in i:
        tmp.append(k.split(":"))
    pcp_list_2.append(tmp)

In [34]:
pcp_list_2[:2]

[[['5799347067982556520', '-1'],
  ['509660095530134768', '-1'],
  ['5755694407684602296', '-1'],
  ['8277336076276184272', '9148482949976129397'],
  ['7908382889764677758', '-1']],
 [['5799347067982556520', '9172976955054793469'],
  ['7908382889764677758',
   '1787573075717641245,9172976955054793469,5195139481388729954,4621934203383159480,9148482949976129397']]]

In [35]:
#每一行的category
each_category_list = []
for i in pcp_list_2:
    tmp = []
    for k in i:
        tmp.append(k[0])
    each_category_list.append(tmp)

In [36]:
each_category_list[:3]

[['5799347067982556520',
  '509660095530134768',
  '5755694407684602296',
  '8277336076276184272',
  '7908382889764677758'],
 ['5799347067982556520', '7908382889764677758'],
 ['5799347067982556520', '7258015885215914736', '7908382889764677758']]

In [37]:
dataframe_pred_cate = doc2vec(each_category_list, 5)
dataframe_pred_cate.head(3)

,0,1,2,3,4
0,0.023941,-0.072742,0.047830,-0.015585,0.001041
1,-0.098020,-0.066511,-0.087521,-0.071039,0.088695
2,0.167344,0.098940,0.126918,-0.170203,0.058301


In [38]:
dataframe_pred_cate.columns = ["pred_cate1","pred_cate2","pred_cate3",
                               "pred_cate4","pred_cate5"]

In [39]:
data_2 = pd.concat([data_1, dataframe_pred_cate],axis=1)
data_2.head(3)

,instance_id,item_id,item_category_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,...,prop6,prop7,prop8,prop9,prop10,pred_cate1,pred_cate2,pred_cate3,pred_cate4,pred_cate5
0,108641074714126964,3412720377098676069,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,4505772604969228686,...,-0.199947,-0.046342,0.088241,0.014969,-0.085267,0.023941,-0.072742,0.047830,-0.015585,0.001041
1,5754713551599725161,3412720377098676069,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,2692638157208937547,...,-0.016272,0.050968,0.010247,0.236364,-0.291694,-0.098020,-0.066511,-0.087521,-0.071039,0.088695
2,842679481291040981,3412720377098676069,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,5247924392014515924,...,0.193300,-0.139771,-0.211507,-0.057105,-0.248570,0.167344,0.098940,0.126918,-0.170203,0.058301


In [40]:
data_2.shape

(496482, 40)

In [41]:
#每一行的property
each_property_list = []
for i in pcp_list_2:
    tmp = []
    for k in i:
        tmp.extend(k[1:])
    each_property_list.append(tmp)

In [42]:
each_property_list[:2]

[['-1', '-1', '-1', '9148482949976129397', '-1'],
 ['9172976955054793469',
  '1787573075717641245,9172976955054793469,5195139481388729954,4621934203383159480,9148482949976129397']]

In [43]:
def property_clean(doc):
    doc_trans = []
    for i in range(len(doc)):
        temp = []
        for j in range(len(doc[i])):
            for k in range(len(doc[i][j].split(","))):
                temp.append(doc[i][j].split(",")[k])
        doc_trans.append(temp)
    return doc_trans

In [44]:
property_clean_1 = property_clean(each_property_list)
property_clean_1[:2]

[['-1', '-1', '-1', '9148482949976129397', '-1'],
 ['9172976955054793469',
  '1787573075717641245',
  '9172976955054793469',
  '5195139481388729954',
  '4621934203383159480',
  '9148482949976129397']]

In [45]:
each_property_list_prep = [list(set(x)) for x in property_clean_1]
each_property_list_prep[:2]

[['9148482949976129397', '-1'],
 ['9148482949976129397',
  '5195139481388729954',
  '4621934203383159480',
  '1787573075717641245',
  '9172976955054793469']]

In [46]:
dataframe_pred_prop = doc2vec(each_property_list_prep, 10)
dataframe_pred_prop.head(3)

,0,1,2,3,4,5,6,7,8,9
0,-0.087757,0.152223,-0.224348,0.068927,-0.110023,-0.021113,-0.184340,0.532930,-0.222828,0.020902
1,-0.213730,-0.006777,-0.040061,-0.092147,-0.126459,0.002954,-0.030236,0.002456,-0.190004,0.070656
2,-0.062296,-0.013683,-0.030650,-0.054671,-0.029541,-0.003900,0.029093,0.019030,0.055204,-0.015705


In [47]:
dataframe_pred_prop.columns = ["pred_prop1","pred_prop2","pred_prop3","pred_prop4","pred_prop5",
                              "pred_prop6","pred_prop7","pred_prop8","pred_prop9","pred_prop10",]
data_3 = pd.concat([data_2, dataframe_pred_prop],axis=1)
data_3.head(3)

,instance_id,item_id,item_category_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,...,pred_prop1,pred_prop2,pred_prop3,pred_prop4,pred_prop5,pred_prop6,pred_prop7,pred_prop8,pred_prop9,pred_prop10
0,108641074714126964,3412720377098676069,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,4505772604969228686,...,-0.087757,0.152223,-0.224348,0.068927,-0.110023,-0.021113,-0.184340,0.532930,-0.222828,0.020902
1,5754713551599725161,3412720377098676069,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,2692638157208937547,...,-0.213730,-0.006777,-0.040061,-0.092147,-0.126459,0.002954,-0.030236,0.002456,-0.190004,0.070656
2,842679481291040981,3412720377098676069,5799347067982556520,1975590437749032870,3948283326616421003,3,3,4,14,5247924392014515924,...,-0.062296,-0.013683,-0.030650,-0.054671,-0.029541,-0.003900,0.029093,0.019030,0.055204,-0.015705


In [48]:
data_3.shape

(496482, 50)

In [49]:
data_3.to_csv("data_nlp.csv")

-----

# context_timestamp特征处理

In [50]:
data_3 = pd.read_csv("data_nlp.csv")

In [51]:
def time_value(time_stamp):
    hour = int(time_stamp.strftime("%H:%M:%S")[:2]) * 3600
    minus = int(time_stamp.strftime("%H:%M:%S")[3:5]) * 60
    sec = int(time_stamp.strftime("%H:%M:%S")[6:8])
    return hour + minus + sec

In [52]:
import datetime 
time_value_list = []
for i in range(len(data_3['context_timestamp'])):
    time_stamp = datetime.datetime.utcfromtimestamp(data_3['context_timestamp'][i])
    time_value_list.append(time_value(time_stamp))

In [53]:
data_3['context_timestamp'] = time_value_list
data_3['context_timestamp'].head(3)

0     7744
1    14432
2    68652
Name: context_timestamp, dtype: int64

In [54]:
data_4 = data_3.copy(deep = True)

In [55]:
#timestamp bin处理
data_4['context_timestamp'] = pd.cut(list(data_4['context_timestamp']),24)

In [56]:
data_4.columns

Index(['Unnamed: 0', 'instance_id', 'item_id', 'item_category_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'prop1', 'prop2', 'prop3', 'prop4', 'prop5',
       'prop6', 'prop7', 'prop8', 'prop9', 'prop10', 'pred_cate1',
       'pred_cate2', 'pred_cate3', 'pred_cate4', 'pred_cate5', 'pred_prop1',
       'pred_prop2', 'pred_prop3', 'pred_prop4', 'pred_prop5', 'pred_prop6',
       'pred_prop7', 'pred_prop8', 'pred_prop9', 'pred_prop10'],
      dtype='object')

In [57]:
data_4.shape

(496482, 51)

In [58]:
#去掉无用的feature
data_5 = data_4.drop(['Unnamed: 0', 'instance_id', 'item_id','user_id','context_id',
                        'item_brand_id','shop_id'],axis = 1)

In [59]:
data_5.shape

(496482, 44)

In [60]:
feature_dummy = ["context_timestamp","user_occupation_id","user_age_level",
                "user_gender_id",'item_city_id','item_category_list']

In [61]:
data_5["user_occupation_id"] = data_5["user_occupation_id"].astype(str)
data_5["user_age_level"] = data_5["user_age_level"].astype(str)
data_5["user_gender_id"] = data_5["user_gender_id"].astype(str)
data_5['item_city_id'] = data_5['item_city_id'].astype(str)
data_5['item_category_list'] = data_5['item_category_list'].astype(str)

In [62]:
data_6 = pd.concat([data_5,pd.get_dummies(data_5[feature_dummy])],axis =1)
data_6 = data_6.drop(["context_timestamp","user_occupation_id","user_age_level",
                "user_gender_id",'item_city_id'],axis = 1)

In [63]:
data_6.shape

(496482, 222)

In [64]:
feature_std = ['item_price_level','shop_score_description','shop_score_delivery',
              'shop_score_service','shop_star_level','shop_review_positive_rate',
              'shop_review_num_level','context_page_id','user_star_level',
               'item_pv_level','item_collected_level','item_sales_level',
              'item_price_level']

In [65]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler() 
std.fit(data_6[feature_std].values)
data_6[feature_std] = std.transform(list(data_6[feature_std].values))       
print(data_6.shape)

(496482, 222)


In [66]:
data_6.columns

Index(['item_category_list', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_star_level',
       'context_page_id', 'predict_category_property', 'shop_review_num_level',
       'shop_review_positive_rate',
       ...
       'item_category_list_2436715285093487584',
       'item_category_list_2642175453151805566',
       'item_category_list_3203673979138763595',
       'item_category_list_4879721024980945592',
       'item_category_list_509660095530134768',
       'item_category_list_5755694407684602296',
       'item_category_list_5799347067982556520',
       'item_category_list_7258015885215914736',
       'item_category_list_8277336076276184272',
       'item_category_list_8710739180200009128'],
      dtype='object', length=222)

In [67]:
data_6 = data_6.drop(["item_category_list",'predict_category_property'],axis = 1)

In [68]:
data_6.head(3)

,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_star_level,context_page_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,...,item_category_list_2436715285093487584,item_category_list_2642175453151805566,item_category_list_3203673979138763595,item_category_list_4879721024980945592,item_category_list_509660095530134768,item_category_list_5755694407684602296,item_category_list_5799347067982556520,item_category_list_7258015885215914736,item_category_list_8277336076276184272,item_category_list_8710739180200009128
0,-3.475377,-3.1379,-3.28112,-1.444348,0.035294,0.656024,-4.162958,0.435111,-4.310423,1.222849,...,0,0,0,0,0,0,1,0,0,0
1,-3.475377,-3.1379,-3.28112,-1.444348,0.057759,-0.587808,-4.162958,0.435111,-4.310423,1.222849,...,0,0,0,0,0,0,1,0,0,0
2,-3.475377,-3.1379,-3.28112,-1.444348,0.042782,-0.587808,-4.162958,0.435111,-4.310423,1.222849,...,0,0,0,0,0,0,1,0,0,0


------

In [69]:
columns_names = [str(i) for i in range(220)]
data_6.columns = columns_names

In [70]:
train = data_6.loc[:478110,:]
test = data_6.loc[478111:,:]

In [71]:
train.shape

(478111, 220)

In [72]:
test.shape

(18371, 220)

In [73]:
from sklearn import model_selection

In [74]:
# train_x, test_x, train_y, test_y = model_selection.train_test_split(train_6[feature_all], 
#                                                                         train_6[Target], 
#                                                                     random_state = 0)
# print("train_6 Shape: {}".format(train_6.shape))
# print("train_x Shape: {}".format(train_x.shape))
# print("test_x Shape: {}".format(test_x.shape))
# train_x.head()

In [75]:
#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, \
                    ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

In [146]:
MLA = [
#     ensemble.AdaBoostClassifier(),
#     ensemble.BaggingClassifier(),
#     ensemble.ExtraTreesClassifier(),
#     ensemble.GradientBoostingClassifier(),
#     ensemble.RandomForestClassifier(),
#     gaussian_process.GaussianProcessClassifier(),
#     linear_model.LogisticRegressionCV(),
#     linear_model.PassiveAggressiveClassifier(),
#     linear_model.RidgeClassifierCV(),
#     linear_model.SGDClassifier(),
#     linear_model.Perceptron(),
#     svm.SVC(probability=True),
#     svm.NuSVC(probability=True),
    LGBMClassifier(num_leaves=63, max_depth=7, n_estimators=80, n_jobs=20)
    ]

In [147]:
MLA

[LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
         learning_rate=0.1, max_depth=7, min_child_samples=20,
         min_child_weight=0.001, min_split_gain=0.0, n_estimators=80,
         n_jobs=20, num_leaves=63, objective=None, random_state=None,
         reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
         subsample_for_bin=200000, subsample_freq=1)]

In [148]:
train = train.reset_index(drop = True)
test = test.reset_index(drop = True)

In [149]:
cv_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .7, random_state = 0 ) 

MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train log_loss Mean', 'MLA Test log_loss Mean', 'MLA Test Accuracy 3*STD' ,'MLA Time']
MLA_compare = pd.DataFrame(columns = MLA_columns)
row_index = 0
for alg in MLA:
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
    cv_results = model_selection.cross_validate(alg, train, 
                                                train_y,cv  = cv_split,
                                                scoring='neg_log_loss')

    MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
    MLA_compare.loc[row_index, 'MLA Train log_loss Mean'] = -cv_results['train_score'].mean()
    MLA_compare.loc[row_index, 'MLA Test log_loss Mean'] = -cv_results['test_score'].mean()   
    MLA_compare.loc[row_index, 'MLA Test log_loss 3*STD'] = -cv_results['test_score'].std()*3   #let's know the worst that can happen!
    
    alg.fit(train, train_y)
    MLA_predict = alg.predict(test)
    print(MLA_name,"complete...")
    
    row_index+=1

MLA_compare.sort_values(by = ['MLA Test log_loss Mean'], ascending = False, inplace = True)
MLA_compare

i:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


LGBMClassifier complete...


i:\anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


,MLA Name,MLA Parameters,MLA Train log_loss Mean,MLA Test log_loss Mean,MLA Test Accuracy 3*STD,MLA Time,MLA Test log_loss 3*STD
0,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",0.0788973,0.0894817,NaN,19.4375,-0.003817


In [150]:
predict_prob = alg.predict_proba(test)[:,1]

In [151]:
submission = pd.read_table("round1_ijcai_18_result_demo_20180301.txt")

In [152]:
submission_final = pd.DataFrame(columns=submission.columns)

In [153]:
submission_final["instance_id"] = test_1["instance_id"]
submission_final["predicted_score"] = predict_prob
submission_final = submission_final.drop("instance_id predicted_score",axis = 1)
submission_final

,instance_id,predicted_score
0,2475218615076601065,0.008389
1,398316874173557226,0.032361
2,6586402638209028583,0.016558
3,1040996105851528465,0.020511
4,6316278569655873454,0.019470
5,868158305045921978,0.019489
6,5713520501786699854,0.012267
7,932945015407923184,0.009208
8,1919197847086752313,0.009198
9,304887065966615346,0.008201


In [154]:
from datetime import date

In [155]:
today = date.today().strftime('%Y%m%d')

In [156]:
submission_final.to_csv("submission_"+today+".csv",index = False,sep = ' ')

In [157]:
submission_final

,instance_id,predicted_score
0,2475218615076601065,0.008389
1,398316874173557226,0.032361
2,6586402638209028583,0.016558
3,1040996105851528465,0.020511
4,6316278569655873454,0.019470
5,868158305045921978,0.019489
6,5713520501786699854,0.012267
7,932945015407923184,0.009208
8,1919197847086752313,0.009198
9,304887065966615346,0.008201
